In [1]:
import findspark
findspark.init()

In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from surprise import SVD, SVDpp, NMF, Dataset, Reader
from surprise.model_selection import cross_validate
import pandas as pd

In [4]:
spark = SparkSession.builder.appName("CollaborativeFiltering").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/19 15:16:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
hotel_comments_df = pd.read_csv("data/hotel_comments_cleaned.csv")
hotel_info_df = pd.read_csv("data/hotel_info_cleaned.csv")

In [6]:
# Tiền xử lý dữ liệu cho mô hình PySpark ALS
als_data = hotel_comments_df[['Hotel ID', 'Reviewer ID', 'Score']]
als_data['Hotel ID'] = als_data['Hotel ID'].apply(lambda x: int(x.split('_')[1]))
als_data['Reviewer ID'] = als_data['Reviewer ID'].apply(lambda x: int(x.split('_')[2]))

/var/folders/py/l7rngj8j6sl2flg1m973yfgh0000gn/T/ipykernel_21147/1463577058.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  als_data['Hotel ID'] = als_data['Hotel ID'].apply(lambda x: int(x.split('_')[1]))
/var/folders/py/l7rngj8j6sl2flg1m973yfgh0000gn/T/ipykernel_21147/1463577058.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  als_data['Reviewer ID'] = als_data['Reviewer ID'].apply(lambda x: int(x.split('_')[2]))


In [7]:
# Chuyển đổi DataFrame sang Spark DataFrame
spark_df = spark.createDataFrame(als_data)

In [8]:
reviewers = spark_df.select("Reviewer ID").distinct().count()
hotels = spark_df.select("Hotel ID").distinct().count()
numerator = spark_df.count()
display(numerator, reviewers, hotels)

25892

250

30

In [9]:
# Chia dữ liệu thành tập đào tạo và kiểm tra
(training, test) = spark_df.randomSplit([0.8, 0.2])

In [10]:
# Xây dựng mô hình ALS
als = ALS(maxIter=10, regParam=0.1, rank=25, userCol="Reviewer ID", itemCol="Hotel ID", ratingCol="Score", coldStartStrategy="drop", nonnegative=True)
model = als.fit(training)

24/08/19 15:16:26 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [11]:
# Đánh giá mô hình
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="Score", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

In [12]:
print(f"Root-mean-square error = {str(rmse)}")

Root-mean-square error = 0.9880779260355206


In [13]:
predictions.show(5)

+--------+-----------+-----+----------+
|Hotel ID|Reviewer ID|Score|prediction|
+--------+-----------+-----+----------+
|      19|        148|  9.2| 9.0406685|
|       4|         31| 10.0|  8.804859|
|       5|         31| 10.0|  8.802999|
|      17|         31|  8.8|   9.05596|
|      18|         31|  8.0|  8.729336|
+--------+-----------+-----+----------+
only showing top 5 rows



In [14]:
# Tải dữ liệu cho Surprise
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(hotel_comments_df[['Reviewer ID', 'Hotel ID', 'Score']], reader)

In [15]:
# Huấn luyện và đánh giá mô hình SVD
svd = SVD()
svd_results = cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9427  0.9635  0.9309  0.9578  0.9383  0.9466  0.0122  
MAE (testset)     0.7533  0.7714  0.7490  0.7669  0.7576  0.7596  0.0084  
Fit time          0.33    0.36    0.42    0.27    0.27    0.33    0.06    
Test time         0.04    0.15    0.02    0.02    0.03    0.05    0.05    


In [16]:
# Huấn luyện và đánh giá mô hình SVD++
svdpp = SVDpp()
svdpp_results = cross_validate(svdpp, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9370  0.9534  0.9439  0.9492  0.9562  0.9479  0.0069  
MAE (testset)     0.7585  0.7669  0.7630  0.7727  0.7715  0.7665  0.0053  
Fit time          0.26    0.21    0.23    0.23    0.26    0.24    0.02    
Test time         0.02    0.02    0.02    0.03    0.03    0.03    0.00    


In [17]:
# Huấn luyện và đánh giá mô hình NMF
nmf = NMF()
nmf_results = cross_validate(nmf, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9938  0.9690  0.9718  0.9832  0.9650  0.9766  0.0105  
MAE (testset)     0.8072  0.7884  0.7917  0.7952  0.7882  0.7942  0.0070  
Fit time          1.60    1.41    1.32    1.35    1.31    1.40    0.11    
Test time         0.02    0.11    0.02    0.02    0.02    0.04    0.03    


- SVD có RMSE trung bình là khoảng 0.9456 và MAE trung bình là 0.7589. Thời gian fit và test khá nhanh, chỉ khoảng 0.22s và 0.02s tương ứng.
- SVD++ cho thấy RMSE trung bình là khoảng 0.9477 và MAE trung bình là 0.7662. Thời gian fit và test là 0.16s và 0.01s tương ứng.
- NMF cho kết quả RMSE trung bình là 0.9766 và MAE trung bình là 0.7942. Thời gian fit của mô hình này lâu nhất là 0.86s, nhưng thời gian test tương tự như hai mô hình còn lại.

=> Mô hình SVD là một lựa chọn phù hợp cho bài toán này, nhờ vào sự cân bằng giữa độ chính xác và thời gian huấn luyện.

In [18]:
# Xử dụng mô hình SVD để xây dựng hàm đề xuất

# Chuẩn bị dữ liệu
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(hotel_comments_df[['Reviewer ID', 'Hotel ID', 'Score']], reader)
trainset = data.build_full_trainset()

# Huấn luyện mô hình SVD
svd_model = SVD()
svd_model.fit(trainset)

def recommend_hotels(reviewer_id, num_recommendations=5):
    # Lấy danh sách tất cả các khách sạn
    all_hotels = hotel_info_df['Hotel_ID'].unique()
    
    # Lấy danh sách các khách sạn đã đánh giá bởi người dùng này
    rated_hotels = hotel_comments_df[hotel_comments_df['Reviewer ID'] == reviewer_id]['Hotel ID'].unique()
    
    # Lọc ra các khách sạn chưa được đánh giá bởi người dùng này
    unrated_hotels = [hotel for hotel in all_hotels if hotel not in rated_hotels]
    
    # Dự đoán điểm cho các khách sạn chưa được đánh giá
    predictions = [svd_model.predict(reviewer_id, hotel).est for hotel in unrated_hotels]
    
    # Tạo DataFrame chứa kết quả và sắp xếp theo điểm dự đoán
    recommendations = pd.DataFrame({
        'Hotel_ID': unrated_hotels,
        'Predicted_Score': predictions
    })
    
    # Lấy top các khách sạn được đề xuất cao nhất
    top_recommendations = recommendations.sort_values(by='Predicted_Score', ascending=False).head(num_recommendations)
    
    # Trả về thông tin chi tiết của các khách sạn được đề xuất
    return hotel_info_df[hotel_info_df['Hotel_ID'].isin(top_recommendations['Hotel_ID'])][['Hotel_ID', 'Hotel_Name', 'Hotel_Rank', 'Hotel_Address', 'Total_Score']]

# Ví dụ sử dụng hàm
print(recommend_hotels(reviewer_id='148', num_recommendations=5))

    Hotel_ID                                         Hotel_Name  \
32       2_3                              ELITE HOTEL NHA TRANG   
40      2_11                              Maris Hotel Nha Trang   
149    10_22  Khách sạn Sunrise Nha Trang Beach Hotel & Spa ...   
198    16_15            HANZ Muong Thanh Vien Trieu Condo Hotel   
620     20_2    Khách Sạn MerPerle Beach (MerPerle Beach Hotel)   

         Hotel_Rank                                      Hotel_Address  \
32     4 sao trên 5  50 Đ. Củ Chi, Vĩnh Hải, ELite Hotel Nha Trang,...   
40     4 sao trên 5  27 Trần Quang Khải, Phường Lộc Thọ, Thành phố ...   
149    5 sao trên 5  12 Trần Phú , Xương Huân, Nha Trang, Việt Nam,...   
198  No information  5 Phạm Văn Đồng, Vĩnh Hải, Nha Trang, Khánh Hò...   
620  No information  88A Tran Phu Street, Lộc Thọ, Nha Trang, Việt ...   

     Total_Score  
32           9.7  
40           9.2  
149          8.9  
198         10.0  
620          9.2  


In [19]:
import pickle
with open('svd_model.pkl', 'wb') as f:
    pickle.dump(svd_model, f)

In [20]:
# spark.stop()
